In [2]:
import os
import pandas as pd
import requests_cache
from retry_requests import retry
from openmeteo_requests import Client
from datetime import datetime
from tqdm import tqdm

# Orijinal ID - koordinat dataframe'i (örn: eğitim/veri kaynağından)
original_df = pd.read_csv("train.csv")  # <-- buraya dosya adını yaz

# Hava kalitesi değişkenleri
air_quality_vars = [
    "pm10", "pm2_5", "carbon_monoxide", "nitrogen_dioxide",
    "sulphur_dioxide", "ozone", "dust", "uv_index"
]

# API setup
cache_session = requests_cache.CachedSession('.cache_air', expire_after=-1)
retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
openmeteo = Client(session=retry_session)

# Klasördeki her dosya için işlem
for file in tqdm(sorted(os.listdir("weather_outputs"))):
    path = os.path.join("weather_outputs", file)
    if not file.endswith(".csv"):
        continue

    df = pd.read_csv(path)

    # Eğer air quality sütunları zaten varsa atla
    if any(col in df.columns for col in air_quality_vars):
        continue

    try:
        # ID'yi dosya adından al
        id_float = float(file.replace("id_", "").replace(".csv", ""))
        coord_row = original_df[original_df["id"] == id_float]
        if coord_row.empty:
            print(f"[{file}] ID orijinal tabloda bulunamadı.")
            continue

        lat = float(coord_row["latitude"].iloc[0])
        lon = float(coord_row["longitude"].iloc[0])

        # Tarihler
        df["date"] = pd.to_datetime(df["date"]).dt.tz_localize(None)
        start_date = df["date"].min().strftime("%Y-%m-%d")
        end_date = df["date"].max().strftime("%Y-%m-%d")

        # API parametreleri
        params = {
            "latitude": lat,
            "longitude": lon,
            "start_date": start_date,
            "end_date": end_date,
            "hourly": air_quality_vars,
            "timezone": "auto"
        }

        responses = openmeteo.weather_api("https://air-quality-api.open-meteo.com/v1/air-quality", params=params)
        response = responses[0]
        hourly = response.Hourly()

        # Tarihleri hazırla
        dates = pd.date_range(
            start=pd.to_datetime(hourly.Time(), unit="s", utc=True),
            end=pd.to_datetime(hourly.TimeEnd(), unit="s", utc=True),
            freq=pd.Timedelta(seconds=hourly.Interval()),
            inclusive="left"
        ).tz_convert(None)

        aq_data = {"date": dates}
        for i, var in enumerate(air_quality_vars):
            aq_data[var] = hourly.Variables(i).ValuesAsNumpy()

        aq_df = pd.DataFrame(aq_data)

        # Merge et (sadece yeni sütunlar eklenecek)
        merged = pd.merge(df, aq_df, on="date", how="left")

        # Aynı dosyanın üstüne yaz
        merged.to_csv(path, index=False)

    except Exception as e:
        print(f"[{file}] Hata oluştu: {e}")


100%|██████████| 7636/7636 [00:16<00:00, 474.71it/s]
